* 紀錄非空的起訴書 non_empty[file_name] = [indictment_key_list]
* 紀錄竊盜罪的起訴書 criminal_320[file_name] = [indictment_key_list]
* 統計總共起訴書數量
* 統計各法條數量

In [15]:
%%time
import os, json, pickle
import regex as re

# transform the word from fullwidth to halfwidth
def transFull2Half(string):
    ans = ''
    for s in string:
        inside_code = ord(s)
        if (inside_code >= 65281 and inside_code <= 65374):
            inside_code -= 65248
        ans += chr(inside_code)
    return ans

indictment_count = 0
laws = {}
non_empty = {}
criminal_320 = {}

filelist = os.listdir('/db1/MOJ_indictments_new/indictment_parsed/')
law_pattern = '(?:：|。|:|所犯法條|參考法條|違反|另犯|請依|全文|按|再|又|而)+(\p{Han}*(法|條例)第\d*條)'

for file in filelist:
    if file.endswith('.json'):
        with open('/db1/MOJ_indictments_new/indictment_parsed/' + file, 'r') as f:
            tmp = json.load(f)
            
            try:
                if tmp['query_result'] != []:
                    indictment_count += tmp['query_result'][0]['num']

                    for key in tmp['details']:
                        clean = re.sub('<\/?(?:div|span|br)[^>]*>|\s|&nbsp;', '', tmp['details'][key]['fulltext'])
                        clean = re.search('(?:書記官|檢察官)(.*)(。附表)?', clean).group(1)
                        result = [match.group(1) for match in re.finditer(law_pattern, clean)]

                        # count for each law
                        for law in result:
                            law = transFull2Half(law)
                            if re.search('^刑法', law): law = '中華民國' + law
                            laws[law] = laws.get(law, 0) + 1

                            if law == '中華民國刑法第320條': 
                                if file not in criminal_320.keys(): criminal_320[file] = []
                                criminal_320[file].append(key)

                        if file not in non_empty.keys(): non_empty[file] = []
                        non_empty[file].append(key)
            except:
                continue

with open('./pkl/non_empty.pkl', 'wb') as f:
    pickle.dump(non_empty, f)
with open('./pkl/criminal_320.pkl', 'wb') as f:
    pickle.dump(criminal_320, f)
with open('./pkl/laws_count.pkl', 'wb') as f:
    pickle.dump(laws, f)
    
print(
f"""total files : {len(filelist)}
non-empty files : {len(non_empty)} 
actually have : {sum([len(v) for k,v in non_empty.items()])}
should have : {indictment_count}

320 files : {len(criminal_320)}
320 indictment : {sum([len(v) for k,v in criminal_320.items()])}
"""
)

total files : 23100
non-empty files : 6753 
actually have : 214914
should have : 214973

320 files : 3231
320 indictment : 18914

CPU times: user 2min 15s, sys: 7.04 s, total: 2min 22s
Wall time: 4min 12s


In [2]:
import pickle
with open('./pkl/laws_count.pkl', 'rb') as f:
    laws = pickle.load(f)
sorted_laws = sorted(laws.items(), key=lambda kv:kv[1], reverse=True)
for law in sorted_laws[:10]:
    print(law)

('中華民國刑法第185條', 62186)
('中華民國刑法第47條', 56157)
('毒品危害防制條例第10條', 43194)
('中華民國刑法第38條', 30395)
('中華民國刑法第320條', 18914)
('中華民國刑法第339條', 18154)
('中華民國刑法第284條', 15662)
('中華民國刑法第55條', 11694)
('毒品危害防制條例第18條', 10098)
('中華民國刑法第30條', 8380)
